Final Model from scratch

In [1]:
# dataset_dir = 'training_data_final'
# !pip install split-folders
# import splitfolders
# splitfolders.ratio("training_data_final", "ok", seed=127, ratio=(0.8, 0.2, 0))

In [5]:
import os
import numpy as np
import tensorflow as tf

tfk = tf.keras
tfkl = tf.keras.layers

In [6]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('model_16_11_split')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=True, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [7]:
# Random seed for reproducibility
seed = 127
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch = 32

data_gen = ImageDataGenerator(
                              width_shift_range=10,
                              vertical_flip=True, 
                              fill_mode='reflect',
                              channel_shift_range=20,
                              rescale=1/255.)

validation_gen = ImageDataGenerator(rescale=1/255.)


train_gen = data_gen.flow_from_directory(directory="ok/train",
                                                           target_size=(96,96),
                                                           color_mode='rgb',
                                                           classes=None,
                                                           class_mode='categorical',
                                                           batch_size=batch,
                                                           shuffle=True,
                                                           seed=seed) # set as training data

val_gen = validation_gen.flow_from_directory(directory="ok/val",
                                                           target_size=(96,96),
                                                           color_mode='rgb',
                                                           classes=None,
                                                           class_mode='categorical',
                                                           batch_size=batch,
                                                           shuffle=True,
                                                           seed=seed) # set as validation data

Found 2829 images belonging to 8 classes.
Found 706 images belonging to 8 classes.


In [9]:
def build_model(input_shape):
    tf.random.set_seed(seed)

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        filters=24,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(input_layer)

    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(conv1)

    pool2 = tfkl.MaxPooling2D(pool_size = (2, 2))(conv2)

    bn = tfkl.BatchNormalization()(pool2)

    conv3 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(bn)

    conv4 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(conv3)
    
    conv5 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(conv4)      

    pool5 = tfkl.MaxPooling2D(pool_size = (2, 2))(conv5)

    bn = tfkl.BatchNormalization()(pool5)

    conv6 = tfkl.Conv2D(
        filters=96,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(bn)

    pool6 = tfkl.MaxPooling2D(pool_size = (2, 2))(conv6)  

    bn = tfkl.BatchNormalization()(pool6)  
    
    conv7 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(bn)

    global_pool = tfkl.GlobalMaxPooling2D()(conv7)

    bn = tfkl.BatchNormalization()(global_pool) 

    dropout = tfkl.Dropout(0.2, seed=seed)(bn)

    classifier_layer = tfkl.Dense(
        units=256, 
        name='Classifier', 
        activation='relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(dropout)

    dropout = tfkl.Dropout(0.2, seed=seed)(classifier_layer)

    classifier_layer2 = tfkl.Dense(
        units=256, 
        name='Classifier2', 
        activation='relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(dropout)

    dropout = tfkl.Dropout(0.2, seed=seed)(classifier_layer2)
    
    output_layer = tfkl.Dense(
        units=8, 
        activation='softmax', 
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name='Output'
    )(dropout)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [10]:
input_shape = (96, 96, 3)

In [11]:
model = build_model(input_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 96, 96, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 24)        672       
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 32)        6944      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 64)        18496 

In [12]:
# create class weights
class_distribution = []
for i in range(8):
  class_distribution.append(sum(1 for x in train_gen.labels if x == i))
  
weights = []
total = train_gen.labels.size

for i in range(8):
  weights.append((i,(1/class_distribution[i]) * (total/8.0)))

class_weight  = {c:weight for (c,weight) in weights}

In [13]:
# Create folders and callbacks and fit
callbacks = create_folders_and_callbacks(model_name='CNN_17_11_1')

epochs = 200
# Train the model
history = model.fit(
    x = train_gen,
    epochs = epochs,
    validation_data = val_gen,
    class_weight = class_weight,
    callbacks = callbacks,
).history

Epoch 1/200
89/89 [==============================] - 78s 821ms/step - loss: 2.1606 - accuracy: 0.2891 - val_loss: 2.0037 - val_accuracy: 0.2068
Epoch 2/200
89/89 [==============================] - 77s 861ms/step - loss: 1.7282 - accuracy: 0.3517 - val_loss: 1.7283 - val_accuracy: 0.2975
Epoch 3/200
89/89 [==============================] - 80s 899ms/step - loss: 1.5714 - accuracy: 0.4100 - val_loss: 1.8625 - val_accuracy: 0.2904
Epoch 4/200
89/89 [==============================] - 77s 865ms/step - loss: 1.4622 - accuracy: 0.4592 - val_loss: 1.3268 - val_accuracy: 0.5142
Epoch 5/200
89/89 [==============================] - 75s 845ms/step - loss: 1.3944 - accuracy: 0.4783 - val_loss: 1.4593 - val_accuracy: 0.4547
Epoch 6/200
89/89 [==============================] - 74s 834ms/step - loss: 1.3480 - accuracy: 0.5072 - val_loss: 1.2529 - val_accuracy: 0.5297
Epoch 7/200
89/89 [==============================] - 76s 856ms/step - loss: 1.2890 - accuracy: 0.5338 - val_loss: 1.2553 - val_accuracy:

In [14]:
model.save("Cnn_best_weights/Cnn_best_weights")

INFO:tensorflow:Assets written to: Cnn_best_weights/Cnn_best_weights\assets


INFO:tensorflow:Assets written to: Cnn_best_weights/Cnn_best_weights\assets


In [17]:
# Evaluate on validation set

model_test_metrics = model.evaluate(val_gen, return_dict=True)

print()
print("Test metrics with weights and data augmentation")
print(model_test_metrics)

23/23 [==============================] - 2s 68ms/step - loss: 0.8181 - accuracy: 0.8130

Test metrics without data augmentation
{'loss': 0.8181033134460449, 'accuracy': 0.8130311369895935}
